In [1]:

import os, json
import numpy as np, pandas as pd

from pprint import pprint
from datetime import datetime as dt


In [6]:

csv_location = 'C:\\Users\\karan\\Documents\\Cricket-Analytics\\IPL\\Data\\ipl_json_to_csv'


In [7]:

print(csv_location)
print()
print(dt.now().date())


C:\Users\karan\Documents\Cricket-Analytics\IPL\Data\ipl_json_to_csv

2023-04-19


In [16]:

folder_path = csv_location

# Get a list of all files and directories in the folder
all_files_and_dirs = os.listdir(folder_path)

# Filter the list to include only files (exclude directories)
files_only = [f for f in all_files_and_dirs if os.path.isfile(os.path.join(folder_path, f))]

# Print total # of files
print(len(files_only))

# Print names of first few files 
pprint(files_only[:5])


19
['1359475_T20_2023_2023_03_31_Chennai Super Kings_v_Gujarat Titans.csv',
 '1359476_T20_2023_2023_04_01_Punjab Kings_v_Kolkata Knight Riders.csv',
 '1359477_T20_2023_2023_04_01_Lucknow Super Giants_v_Delhi Capitals.csv',
 '1359478_T20_2023_2023_04_02_Rajasthan Royals_v_Sunrisers Hyderabad.csv',
 '1359479_T20_2023_2023_04_02_Mumbai Indians_v_Royal Challengers Bangalore.csv']


In [17]:

files_list = files_only
total_files = len(files_list)

#print(np.arange(total_files))
print(total_files)


19


In [10]:

#All file numbers > 1359475 are #2023 IPL onwards
#All file numbers > 1304047 are #2022 IPL onwards


In [18]:

# initialize an empty DataFrame to store all the data
master_df = pd.DataFrame()

# iterate through each file in the directory
for file_name in os.listdir(csv_location):
    if file_name.endswith('.csv'):
        # read the CSV file into a DataFrame
        file_path = os.path.join(csv_location, file_name)
        df = pd.read_csv(file_path)
        
        # append the data to the master DataFrame
        master_df = master_df.append(df)

# print the master DataFrame
print(master_df.shape)


(442, 18)


In [12]:

pprint(master_df.columns.values.tolist())


['inn_bat',
 'name',
 'mnum',
 'rs',
 'four',
 'six',
 'out',
 'bowf',
 'inn_bow',
 'rc',
 'balb',
 'wkt',
 'season',
 'type',
 'date',
 'city',
 'teams',
 'team_nm']


In [21]:

#Variables to help with fantasy pts scoring

# BATTING
master_df['pts_for_thrty'] = master_df['rs'].apply(lambda x : 4 if x >= 30 else 0 )
master_df['pts_for_fifty'] = master_df['rs'].apply(lambda x : 8 if x >= 50 else 0 )
master_df['pts_for_hundy'] = master_df['rs'].apply(lambda x : 16 if x >= 100 else 0 )
master_df['pts_for_ducky'] = master_df['rs'].apply(lambda x : -2 if x == 0 else 0 )


# BOWLING
master_df['pts_for_3w'] = master_df['wkt'].apply(lambda x : 4 if x >= 3 else 0 )
master_df['pts_for_4w'] = master_df['wkt'].apply(lambda x : 8 if x >= 4 else 0 )
master_df['pts_for_5w'] = master_df['wkt'].apply(lambda x : 16 if x >= 5 else 0 )

master_df['bow_econ'] = ( master_df['rc'] * 6 ) / master_df['balb']

master_df['pts_for_bow_econ'] = master_df.apply(lambda x : (2*6)/x.balb if (5 <= x.bow_econ) & ( x.bow_econ < 6) \
                                                            else ( (4*6)/x.balb if (5 <= x.bow_econ) & ( x.bow_econ < 6) else 0 )\
                                                , axis =1)


In [22]:

#Create Fantasy pts crickpe version

master_df['bat_pts'] = master_df['rs'].fillna(0) + master_df['four'].fillna(0) + master_df['six'].fillna(0)*2 \
    + master_df['pts_for_thrty'].fillna(0) + master_df['pts_for_fifty'].fillna(0) + master_df['pts_for_hundy'].fillna(0) + master_df['pts_for_ducky'].fillna(0)

master_df['bow_pts'] = master_df['wkt'].fillna(0)*25 \
    + master_df['pts_for_3w'].fillna(0) + master_df['pts_for_4w'].fillna(0) + master_df['pts_for_5w'].fillna(0) \
        + master_df['pts_for_bow_econ'].fillna(0)

master_df['pts_crickpe'] = master_df['bat_pts'].fillna(0) + master_df['bow_pts'].fillna(0)


In [23]:

display(master_df.head())


,inn_bat,name,mnum,rs,four,six,out,bowf,inn_bow,rc,...,pts_for_hundy,pts_for_ducky,pts_for_3w,pts_for_4w,pts_for_5w,bow_econ,pts_for_bow_econ,bat_pts,bow_pts,pts_crickpe
0,1,AT Rayudu,1,12.0,0.0,1.0,1.0,12.0,NaN,NaN,...,0,0,0,0,0,NaN,0.0,14.0,0.0,14.0
1,1,BA Stokes,1,7.0,1.0,0.0,1.0,6.0,NaN,NaN,...,0,0,0,0,0,NaN,0.0,8.0,0.0,8.0
2,1,DP Conway,1,1.0,0.0,0.0,1.0,6.0,NaN,NaN,...,0,0,0,0,0,NaN,0.0,1.0,0.0,1.0
3,1,MJ Santner,1,1.0,0.0,0.0,0.0,3.0,2.0,32.0,...,0,0,0,0,0,8.0,0.0,1.0,0.0,1.0
4,1,MM Ali,1,23.0,4.0,1.0,1.0,16.0,NaN,NaN,...,0,0,0,0,0,NaN,0.0,29.0,0.0,29.0


In [24]:

exp_location = 'C:\\Users\\karan\\Documents\\Cricket-Analytics\\IPL\\Data\\exp\\'
exp_filename = 'master_df.csv'

master_df.to_csv( exp_location + exp_filename, index = False)
